In [ ]:
import akshare as ak
import pandas as pd
#import datetime
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 行情数据

stock_price = ak.stock_zh_a_spot_em()

In [ ]:
stock_price_8 = stock_price#[stock_price['涨跌幅'].between(9, 40, inclusive=False)]

In [ ]:
stock_price_8

In [ ]:
stock_price_8['流通MV'] = stock_price_8['流通市值']/100000000

stock_price_8 = stock_price_8[['代码','名称','涨跌幅','市盈率-动态','市净率','流通MV']].round(2)

In [ ]:
stock_concepts = pd.read_csv('./static/stock_concepts.csv',encoding="gbk").fillna('')

# stock_concepts = pd.read_csv('./static/stock_concepts_20220909.csv',encoding="gbk").fillna('')

In [ ]:
stock_industry = pd.read_csv('./static/stock_industry.csv',encoding='gbk').fillna('')
stock_industry['代码'] = stock_industry['股票代码'].str.replace('.SH','').str.replace('.SZ','').str.replace('.BJ','')

In [ ]:
#stock_concepts['代码'] = stock_concepts['股票代码'].str.replace('.SH','').str.replace('.SZ','').str.replace('.BJ','')

In [ ]:
result = stock_price_8.merge(stock_concepts,how='left',on=['代码','名称'])\
                      .merge(stock_industry, how = 'left',on=['代码']) .fillna('')
# result = stock_price_8.merge(stock_concepts,how='left',on=['代码']).fillna('')

In [ ]:
#result

In [ ]:
concept_list = []

for row, l in result.iterrows():
    concept_list = concept_list + l['行业概念'].split(',') #+ l['事件概念'].split(',') 
    concept_list =  [x for x in concept_list if x]   # remove ''
hot_concepts = pd.DataFrame(concept_list).apply(pd.value_counts)

In [ ]:
hot_concepts

In [ ]:
concept_list = []

for row, l in result.iterrows():
    concept_list = concept_list + l['交易概念'].split(',') 
    concept_list =  [x for x in concept_list if x]   # remove ''
hot_concepts = pd.DataFrame(concept_list).apply(pd.value_counts)
hot_concepts

In [ ]:
hot_concepts = set(hot_concepts[hot_concepts[0] > 2].index)
hot_concepts

In [ ]:
result['热点概念'] = ''
for row, l in result.iterrows():
    result['热点概念'][row] = ','.join(set(l['概念名称'].split(',')).intersection(hot_concepts))

In [ ]:
result[['代码','名称','涨跌幅','所属行业','行业概念','事件概念','热点概念']]

In [ ]:
stock_price_5 = stock_price[stock_price['涨跌幅'].between(0, 9, inclusive=False)]
stock_price_5['流通MV'] = stock_price_5['流通市值']/100000000
stock_price_5 = stock_price_5[['代码','名称','涨跌幅','市盈率-动态','市净率','流通MV']].round(2)
result = stock_price_5.merge(stock_concepts,how='left',on=['代码','名称'])\
                      .merge(stock_industry, how = 'left',on=['代码']) .fillna('')

result['热点概念'] = ''
for row, l in result.iterrows():
    result['热点概念'][row] = ','.join(set(l['概念名称'].split(',')).intersection(hot_concepts))
result[['代码','名称','涨跌幅','所属行业','行业概念','事件概念','热点概念']]    

In [ ]:
stock_price_5 = stock_price[stock_price['涨跌幅'].between(-40, -7, inclusive=False)]
stock_price_5['流通MV'] = stock_price_5['流通市值']/100000000
stock_price_5 = stock_price_5[['代码','名称','涨跌幅','市盈率-动态','市净率','流通MV']].round(2)
result = stock_price_5.merge(stock_concepts,how='left',on=['代码','名称'])\
                      .merge(stock_industry, how = 'left',on=['代码']) .fillna('')

result['热点概念'] = ''
for row, l in result.iterrows():
    result['热点概念'][row] = ','.join(set(l['概念名称'].split(',')).intersection(hot_concepts))
result[['代码','名称','涨跌幅','所属行业','行业概念','事件概念','热点概念']] 

In [ ]:
stock_price_5 = stock_price#[stock_price['涨跌幅'].between(-5, -2, inclusive=False)]
stock_price_5['流通MV'] = stock_price_5['流通市值']/100000000
stock_price_5 = stock_price_5[['代码','名称','涨跌幅','市盈率-动态','市净率','流通MV']].round(2)
result = stock_price_5.merge(stock_concepts,how='left',on=['代码','名称'])\
                      .merge(stock_industry, how = 'left',on=['代码']) .fillna('')

result['热点概念'] = ''
for row, l in result.iterrows():
    result['热点概念'][row] = ','.join(set(l['概念名称'].split(',')).intersection(hot_concepts))
result[['代码','名称','涨跌幅','所属行业','行业概念','事件概念','热点概念']] 

In [ ]:
ak.stock_zh_index_spot()

In [ ]:
infrastructure = ak.index_stock_cons(symbol="399359")
infra = infrastructure.merge(result, how = 'left', left_on = ['品种代码'], right_on = ['代码'])
infra[['代码','名称','涨跌幅','所属行业','行业概念','事件概念','热点概念']] 

In [ ]:
ak.js_news(timestamp="2022-09-10 00:00:00")

In [ ]:
ak.index_stock_cons(symbol = '000025')

In [ ]:
ak.stock_zh_a_hist(symbol="000001", period="daily", start_date="20020901", end_date='202209010', adjust="hfq")

In [ ]:
result['代码']

In [ ]:
main_business = pd.DataFrame()

for i in result['代码']:
    stock_main_business = ak.stock_zygc_ym(symbol=i)
    stock_mb = stock_main_business[['报告期','分类方向','分类','营业收入-占主营收入比']]
    stock_mb_prod = stock_mb[(stock_mb['报告期'] == '2021年度') & (stock_mb['分类方向'].isin(['按产品分']) )\
             & (~stock_mb['分类'].str.contains('合计|其他|其它')) ] #& (stock_mb['营业收入-占主营收入比'].str.replace('%','').astype(float) > 2)]
    product = ','.join(set(stock_mb_prod['分类'].str.replace('业务','').str.replace('收入','').str.replace('行业','')))
    stock_mb_prod['主营构成'] = stock_mb_prod['分类'] + '-' + stock_mb_prod['营业收入-占主营收入比']
    product_con = ','.join(stock_mb_prod['主营构成'].str.replace('业务','').str.replace('收入','').str.replace('行业',''))

    stock_mb_ind = stock_mb[(stock_mb['报告期'] == '2021年度') & (stock_mb['分类方向'].isin(['按行业分']) )\
             & (~stock_mb['分类'].str.contains('合计|其他|其它')) ] #& (stock_mb['营业收入-占主营收入比'].str.replace('%','').astype(float) > 2)]
    industry = ','.join(set(stock_mb_ind['分类'].str.replace('业务','').str.replace('收入','').str.replace('行业','')))
    stock_mb_ind['主营构成'] = stock_mb_ind['分类'] + '-' + stock_mb_ind['营业收入-占主营收入比']
    industry_con = ','.join(stock_mb_ind['主营构成'].str.replace('业务','').str.replace('收入','').str.replace('行业',''))
    
    data = {'代码' : i ,'产品' : product, '产品构成' : product_con, '行业': industry, '行业构成': industry_con }
    df = pd.DataFrame(data, index=[0])
    main_business = main_business.append(df)
    
    print(i)

main_business.to_csv('stock_main_business.csv', index = False, encoding = 'gbk')

In [ ]:
main_business.to_csv('stock_main_business.csv', index = False)

In [ ]:
set(result['代码']).difference(list(main_business['代码']))

In [ ]:
stock_main_business = ak.stock_zygc_ym(symbol="301020")
stock_mb = stock_main_business[['报告期','分类方向','分类','营业收入-占主营收入比']]
stock_mb_prod = stock_mb[(stock_mb['报告期'] == '2021年度') & (stock_mb['分类方向'].isin(['按产品分']) )\
         & (~stock_mb['分类'].str.contains('合计|其他|其它')) ] #& (stock_mb['营业收入-占主营收入比'].str.replace('%','').astype(float) > 2)]
product = ','.join(set(stock_mb_prod['分类'].str.replace('业务','').str.replace('收入','').str.replace('行业','')))
stock_mb_prod['主营构成'] = stock_mb_prod['分类'] + '-' + stock_mb_prod['营业收入-占主营收入比']
product_con = ','.join(stock_mb_prod['主营构成'].str.replace('业务','').str.replace('收入','').str.replace('行业',''))

stock_mb_ind = stock_mb[(stock_mb['报告期'] == '2021年度') & (stock_mb['分类方向'].isin(['按行业分']) )\
         & (~stock_mb['分类'].str.contains('合计|其他|其它')) ] #& (stock_mb['营业收入-占主营收入比'].str.replace('%','').astype(float) > 2)]
industry = ','.join(set(stock_mb_ind['分类'].str.replace('业务','').str.replace('收入','').str.replace('行业','')))
stock_mb_ind['主营构成'] = stock_mb_ind['分类'] + '-' + stock_mb_ind['营业收入-占主营收入比']
industry_con = ','.join(stock_mb_ind['主营构成'].str.replace('业务','').str.replace('收入','').str.replace('行业',''))
data = {'代码' : '601222' ,'产品' : product, '产品构成' : product_con, '行业': industry, '行业构成': industry_con }
df = pd.DataFrame(data, index=[0])
df 

In [ ]:
stock_mb